# Assignment

In the lab, we saw how we can take a training script and use `mlflow` to submit runs. In practice, the training script isn't always clean code that is ready to use. So in this assignment, we learn to **refactor code** to train a model and prepare it for scoring. We learn about functionality in `sklearn` for **model persistence**, a fancy term for saving a model, so we can later load it and use it for prediction. We also learn how we can chain pre-processing steps and attach them to the model prediction step so we can both pre-process and score in one smooth flow.

The bulk of the code that needs to execute is already given. This should look like code that we've written throughout the course. But when moving to production, it is **highly, highly recommended** that we **refactor** the code. What this means is that we need to go over the code from top to finish and do a bunch of things (now that we have the advantage of **hindsight**):

- add **comments** in the code, for future us or (heaven forbid!) if someone else has to look at our code!
- remove **extra code** that we wrote during development for debugging purposes but no longer need, or at least comment it out
- simplify things, remove redundancies and make the code more **modular** by using functions or classes if needed
- **parametrize** the code, so you avoid **hard-coding** things that need to change, and move them as high-level parameters at the top of the code, making it easy to change things without breaking things
- create a runtime for the environment using Conda or PIP, which acts as a snapshot of the Python libraries we used and pins down their versions (careful: not all packages used during training are needed during scoring, and since the scoring enivironment is supposed to be **lightweight**, we should identify and remove such packages)
- add **scaffolding**, this is to make sure that the code executes **gracefully** when errors happen, such as when the model expects a certain feature in the future data but it is missing for some reason (by gracefully, we mean that we use things like `try` and `except` to catch and redirect errors)
- last but not least, never stop **testing**, but testing here can mean **unit testing**, **integration testing**, and even statistical tests for **data drift** or **model drift**
- if you haven't yet (what have you been waiting for!) begin to **version control** your code using **git** or something similar

Of course even with hindsight, doing these things is not easy and the answers are not always available, but we do the best we can and with experience we get better at it. Every project can be viewed as a work in progress, but applying certain **best practices** can make it easier to keep improving things without breaking them. This is what **agile development** is all about. You may have noticed that all the above steps are things we do generally when we write applications. It's just that not all data scientists have a rigorous computer science background and so we tend to have looser standards in general. The above is just the beginnig, not the end. Usually depending on the type of deployment we are doing, there are specific additional steps needed. 

Enough said. It's time for work! We used our knowledge of data science to write up some code to read data, pre-process it, train a model and use it to get predictions on a test data set. Here's a standard training code snippet. Examine it and run it to make sure it works.

In [3]:
import pandas as pd

bank = pd.read_csv('data/bank-full.csv', delimiter = ';')

num_cols = bank.select_dtypes(['integer', 'float']).columns
cat_cols = bank.select_dtypes(['object']).drop(columns = "y").columns

print("Numeric columns are {}.".format(", ".join(num_cols)))
print("Categorical columns are {}.".format(", ".join(cat_cols)))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bank.drop(columns = "y"), bank["y"], 
                                                    test_size = 0.10, random_state = 42)

X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)

print("Training data has {} rows.".format(X_train.shape[0]))
print("Test data has {} rows.".format(X_test.shape[0]))

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

onehoter = OneHotEncoder(sparse = False)
onehoter.fit(X_train[cat_cols])
onehot_cols = onehoter.get_feature_names(cat_cols)
X_train_onehot = pd.DataFrame(onehoter.transform(X_train[cat_cols]), columns = onehot_cols)
X_test_onehot = pd.DataFrame(onehoter.transform(X_test[cat_cols]), columns = onehot_cols)

znormalizer = StandardScaler()
znormalizer.fit(X_train[num_cols])
X_train_norm = pd.DataFrame(znormalizer.transform(X_train[num_cols]), columns = num_cols)
X_test_norm = pd.DataFrame(znormalizer.transform(X_test[num_cols]), columns = num_cols)

X_train_featurized = X_train_onehot # add one-hot-encoded columns
X_test_featurized = X_test_onehot   # add one-hot-encoded columns
X_train_featurized[num_cols] = X_train_norm # add numeric columns
X_test_featurized[num_cols] = X_test_norm   # add numeric columns

del X_train_norm, X_test_norm, X_train_onehot, X_test_onehot

print("Featurized training data has {} rows and {} columns.".format(*X_train_featurized.shape))
print("Featurized test data has {} rows and {} columns.".format(*X_test_featurized.shape))

from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(max_iter = 5000, solver = 'lbfgs')
logit.fit(X_train_featurized, y_train)

y_hat_train = logit.predict(X_train_featurized)
y_hat_test = logit.predict(X_test_featurized)

from sklearn.metrics import precision_score, recall_score
precision_train = precision_score(y_train, y_hat_train, pos_label = 'yes') * 100
precision_test = precision_score(y_test, y_hat_test, pos_label = 'yes') * 100

recall_train = recall_score(y_train, y_hat_train, pos_label = 'yes') * 100
recall_test = recall_score(y_test, y_hat_test, pos_label = 'yes') * 100

print("Precision = {:.0f}% and recall = {:.0f}% on the training data.".format(precision_train, recall_train))
print("Precision = {:.0f}% and recall = {:.0f}% on the validation data.".format(precision_test, recall_test))

Numeric columns are age, balance, day, duration, campaign, pdays, previous.
Categorical columns are job, marital, education, default, housing, loan, contact, month, poutcome.
Training data has 40689 rows.
Test data has 4522 rows.


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Featurized training data has 40689 rows and 51 columns.
Featurized test data has 4522 rows and 51 columns.
Precision = 65% and recall = 35% on the training data.
Precision = 63% and recall = 34% on the validation data.


The code above trains a model on data that contains both categorical and numeric features. We normalize the numeric features and one-hot-encode the categorical features as part of pre-processing. In the above code we do this "manually", however as shown [here](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html) we can compose data transformations and ML steps to create a **single multi-step pipeline**. The pipeline object (conviniently called `pipeline` in the docs) has a `fit` and `predict` method:
- By calling `fit`, the raw input data is first transformed into the featurized data, and then passed to the ML algorithm to train a model.
- By calling `predict`, the raw input data is first transformed into the featurized data (just like `fit`), and and then used to get predictions (using the model trained when we called `fit`).

You can even create your own transformers and inculde them in a pipeline step, using as shown [here](https://scikit-learn.org/stable/modules/preprocessing.html#custom-transformers), but we won't worry about this for this assignment.

Let's also have some data that we can use for scoring:

In [4]:
%%writefile data/new_data.json
{"age": {"0": 40, "1": 47},
 "balance": {"0": 580, "1": 3644},
 "campaign": {"0": 1, "1": 2},
 "contact": {"0": "unknown", "1": "unknown"},
 "day": {"0": 16, "1": 9},
 "default": {"0": "no", "1": "no"},
 "duration": {"0": 192, "1": 83},
 "education": {"0": "secondary", "1": "secondary"},
 "housing": {"0": "yes", "1": "no"},
 "job": {"0": "blue-collar", "1": "services"},
 "loan": {"0": "no", "1": "no"},
 "marital": {"0": "married", "1": "single"},
 "month": {"0": "may", "1": "jun"},
 "pdays": {"0": -1, "1": -1},
 "poutcome": {"0": "unknown", "1": "unknown"},
 "previous": {"0": 0, "1": 0}}

Overwriting data/new_data.json


Run through the following steps to first refactor and test code:

- **Step 1:** Compose the data processing and training steps in the above code into one pipeline as shown in the referenced doc. <span style="color:red" float:right>[15 point]</span>
- **Step 2:** Call `fit` and `predict` on the pipeline to make sure that it all works. Remember to pass them the **un-processed** (original) data, since the data processing should be built into the pipeline now. <span style="color:red" float:right>[5 point]</span>
- **Step 3:** Save your pipeline object using `joblib` as shown [here](https://sklearn.org/modules/model_persistence.html). <span style="color:red" float:right>[5 point]</span>
- **Step 4:** Now write a **new script** for scoring: it loads the pipeline you saved in the last step, reads the data `../data/new_data.json` and converts it to a `pandas.DataFrame` object, and obtains predictions on it. The predictions should be stored as a `json` file `../data/new_preds.json`. <span style="color:red" float:right>[10 point]</span>

To begin work on the assignemnt, it's best to first copy the training script in a cell below and begin modifying it according to the instructions in steps 1-3. Then create a new cell and populating with the scoring script described in step 4.

In [5]:
pip install category_encoders


  Using cached category_encoders-2.6.1-py2.py3-none-any.whl (81 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## modified training script goes here
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

import pandas as pd

bank = pd.read_csv('data/bank-full.csv', delimiter = ';')

num_cols = bank.select_dtypes(['integer', 'float']).columns
cat_cols = bank.select_dtypes(['object']).drop(columns = "y").columns

print("Numeric columns are {}.".format(", ".join(num_cols)))
print("Categorical columns are {}.".format(", ".join(cat_cols)))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(bank.drop(columns = "y"), bank["y"], 
                                                    test_size = 0.10, random_state = 42)

X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)


Numeric columns are age, balance, day, duration, campaign, pdays, previous.
Categorical columns are job, marital, education, default, housing, loan, contact, month, poutcome.


##### We wanted to create pipeline for organizing our model.
##### we have 3 parts.
##### 1.We normalized the numrical features. and we did it by using standardscaler
##### 2.we hotencoded the categorical features and we did it by using OneHotEncoder\
##### 3.we put these 2 parts in a object called preprocessor 
##### 4.Then we defined a pipeline and we joined the mentioned parts above with our logisticregression classifier.
##### 5.Then we trained our model and did prediction based on the data which is original.
##### 6.We printed the accuracy which is similar to the model we trained without pipeline.


In [7]:
numeric_features = num_cols
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features =cat_cols
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [8]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(bank.drop(columns = "y"), bank["y"], 
                                                    test_size = 0.10, random_state = 42)
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.895


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
y_hat_train_clf = clf.predict(X_train)
y_hat_test_clf = clf.predict(X_test)

from sklearn.metrics import precision_score, recall_score
precision_train = precision_score(y_train, y_hat_train_clf, pos_label = 'yes') * 100
precision_test = precision_score(y_test, y_hat_test_clf, pos_label = 'yes') * 100

recall_train = recall_score(y_train, y_hat_train_clf, pos_label = 'yes') * 100
recall_test = recall_score(y_test, y_hat_test_clf, pos_label = 'yes') * 100

In [10]:
print("Precision = {:.0f}% and recall = {:.0f}% on the training data.".format(precision_train, recall_train))
print("Precision = {:.0f}% and recall = {:.0f}% on the validation data.".format(precision_test, recall_test))

Precision = 65% and recall = 35% on the training data.
Precision = 63% and recall = 34% on the validation data.


##### Then we saved our pipeline for future predictions using dump function in joblib library.

In [11]:
import joblib
joblib.dump(clf,'clf_model.joblib')


['clf_model.joblib']

##### We load our pipeline becuase we wanted to do some prediction on a data which is in json format.

In [12]:
## scoring script goes here
clf_load = joblib.load('clf_model.joblib')

In [13]:
import pandas as pd
import json

Since we have a json data, we load the data with open file

In [14]:
# Opening JSON file
with open('data/new_data.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
    print(json_object)
    print(type(json_object))


json_object 

{'age': {'0': 40, '1': 47}, 'balance': {'0': 580, '1': 3644}, 'campaign': {'0': 1, '1': 2}, 'contact': {'0': 'unknown', '1': 'unknown'}, 'day': {'0': 16, '1': 9}, 'default': {'0': 'no', '1': 'no'}, 'duration': {'0': 192, '1': 83}, 'education': {'0': 'secondary', '1': 'secondary'}, 'housing': {'0': 'yes', '1': 'no'}, 'job': {'0': 'blue-collar', '1': 'services'}, 'loan': {'0': 'no', '1': 'no'}, 'marital': {'0': 'married', '1': 'single'}, 'month': {'0': 'may', '1': 'jun'}, 'pdays': {'0': -1, '1': -1}, 'poutcome': {'0': 'unknown', '1': 'unknown'}, 'previous': {'0': 0, '1': 0}}
<class 'dict'>


{'age': {'0': 40, '1': 47},
 'balance': {'0': 580, '1': 3644},
 'campaign': {'0': 1, '1': 2},
 'contact': {'0': 'unknown', '1': 'unknown'},
 'day': {'0': 16, '1': 9},
 'default': {'0': 'no', '1': 'no'},
 'duration': {'0': 192, '1': 83},
 'education': {'0': 'secondary', '1': 'secondary'},
 'housing': {'0': 'yes', '1': 'no'},
 'job': {'0': 'blue-collar', '1': 'services'},
 'loan': {'0': 'no', '1': 'no'},
 'marital': {'0': 'married', '1': 'single'},
 'month': {'0': 'may', '1': 'jun'},
 'pdays': {'0': -1, '1': -1},
 'poutcome': {'0': 'unknown', '1': 'unknown'},
 'previous': {'0': 0, '1': 0}}

##### For using this data in our modeling, first we change the data to dataframe and did predictions using our pipeline.

In [20]:
#new_data_json = json.loads(new_data)
df=pd.DataFrame.from_dict(json_object)

df

,age,balance,campaign,contact,day,default,duration,education,housing,job,loan,marital,month,pdays,poutcome,previous
0,40,580,1,unknown,16,no,192,secondary,yes,blue-collar,no,married,may,-1,unknown,0
1,47,3644,2,unknown,9,no,83,secondary,no,services,no,single,jun,-1,unknown,0


In [16]:
predictions = clf_load.predict(df)
predictions

array(['no', 'no'], dtype=object)

For saving it as a json file, we change it to list and the used dump function to change it to json file.

In [17]:
list_predict=predictions.tolist()

In [18]:
json_data = json.dumps(list_predict)
json_data
print(json_data)


["no", "no"]


For saving it in a json file format, we used open function to save the file as a json format.

In [19]:
file_path = "json_data.json"

# Write the data to the JSON file
with open(file_path, "w") as json_file:
    json.dump(json_data, json_file)

print("JSON file created successfully.")

JSON file created successfully.
